In [1]:
import torch.nn as nn
import torch
import math

batch_size = 2
image_size = 8
dim_in = 3
emb_dim = 24
c_group = 1
kernel_size = 3
stride = 2
c_group = 1
gamma = 0.5
beta = 0.2

running_mean = torch.zeros(emb_dim)
running_var = torch.zeros(emb_dim)
for i in range(emb_dim):
    running_mean[i] = 8
    running_var[i] = 21.5

image = torch.zeros(batch_size, dim_in, image_size, image_size)
for b in range(batch_size):
    for c in range(dim_in):
        for i in range(image_size):
            for j in range(image_size):
                image[b, c, i, j] = i + c

print('this is the image')
print(image[0, 0, :, :])

kernel = torch.zeros(emb_dim, dim_in, kernel_size, kernel_size)
kernel[:, :, :, :] = 0.1
bias = torch.zeros((emb_dim,))
for c in range(emb_dim):
    bias[c] = c + 0.01

c_group = math.gcd(dim_in, emb_dim) if c_group == -1 else c_group
padding = math.ceil(((kernel_size - 1) * 1 + 1 - stride) / 2)
print('padding: ', padding)
conv = nn.Conv2d(dim_in, emb_dim, kernel_size, stride, padding, dilation=1, groups=c_group, bias=True)
conv.weight.data = kernel
conv.bias.data = bias
image = conv(image)
print('after conv')
print(image[0, 0, :, :])

norm = nn.BatchNorm2d(emb_dim, momentum=0)
norm.running_mean = running_mean
norm.running_var = running_var
norm.weight.data.fill_(gamma)
norm.bias.data.fill_(beta)
norm.eval()
result = norm(image)

print(result.shape)
print(result[0, 0, :, :])

this is the image
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3., 3., 3.],
        [4., 4., 4., 4., 4., 4., 4., 4.],
        [5., 5., 5., 5., 5., 5., 5., 5.],
        [6., 6., 6., 6., 6., 6., 6., 6.],
        [7., 7., 7., 7., 7., 7., 7., 7.]])
padding:  1
after conv
tensor([[ 1.8100,  2.7100,  2.7100,  2.7100],
        [ 5.4100,  8.1100,  8.1100,  8.1100],
        [ 9.0100, 13.5100, 13.5100, 13.5100],
        [12.6100, 18.9100, 18.9100, 18.9100]], grad_fn=<SliceBackward0>)
torch.Size([2, 24, 4, 4])
tensor([[-0.4675, -0.3704, -0.3704, -0.3704],
        [-0.0793,  0.2119,  0.2119,  0.2119],
        [ 0.3089,  0.7942,  0.7942,  0.7942],
        [ 0.6971,  1.3765,  1.3765,  1.3765]], grad_fn=<SliceBackward0>)
